# Import Library

In [ ]:
from run import *
from tint.metrics import mse, mae
import tint, gc
from tqdm import tqdm
import pandas as pd
from utils.explainer import *
from utils.explainer import *

In [ ]:
from captum.attr import (
    DeepLift,
    GradientShap,
    IntegratedGradients,
    Lime
)

from tint.attr import (
    AugmentedOcclusion,
    DynaMask,
    Occlusion, 
    FeatureAblation
)

# Argument Parser

In [ ]:
parser = get_parser()
argv = """
  --root_path ./dataset/illness/ \
  --data_path national_illness.csv \
  --model_id ili_36_24 \
  --model Transformer \
  --data custom \
    --use_gpu
  --features MS \
  --seq_len 36 \
  --label_len 18 \
  --pred_len 24 \
  --e_layers 2 \
  --d_layers 1 \
  --factor 3 \
  --enc_in 7 \
  --dec_in 7 \
  --c_out 7 \
  --des Exp \
  --itr 1
""".split()
args = parser.parse_args(argv)

In [ ]:
args.explainers = [
    "deep_lift",
    "gradient_shap",
    "integrated_gradients",
    "lime",
    "occlusion",
    "augmented_occlusion",
    "dyna_mask"
]

In [ ]:
explainer_name_map = {"deep_lift":DeepLift,
    "gradient_shap":GradientShap,
    "integrated_gradients":IntegratedGradients,
    "lime":Lime,
    "occlusion":Occlusion,
    # "augmented_occlusion":AugmentedOcclusion,
    "dyna_mask":DynaMask,
    "feature_ablation":FeatureAblation
}

# Initialize Experiment

In [ ]:
set_random_seed(args.seed)
# Disable cudnn if using cuda accelerator.
# Please see https://captum.ai/docs/faq#how-can-i-resolve-cudnn-rnn-backward-error-for-rnn-or-lstm-network
# args.use_gpu = False

assert args.task_name == 'long_term_forecast', "Only long_term_forecast is supported for now"
Exp = Exp_Long_Term_Forecast

setting = stringify_setting(args, 0)

In [ ]:
exp = Exp(args)  # set experiments
_, dataloader = exp._get_data('test')
exp.model.load_state_dict(
    torch.load(os.path.join('checkpoints/' + setting, 'checkpoint.pth'))
)
result_folder = './results/' + setting + '/'

# Evaluation

## Model

In [ ]:
model = exp.model
model.eval()
model.zero_grad()

# only need to output targets, sinec interpretation is based on outputs
assert not exp.args.output_attention

## Evaluate

In [ ]:
expl_metric_map = {
    'mae': mae, 'mse': mse
}
areas = [0.1, 0.2, 0.5]

explainers = ['deep_lift', 'feature_ablation'] # explainers = args.explainers
explainers_map = dict()
for name in explainers:
    explainers_map[name] = explainer_name_map[name](model)

In [ ]:
results = []
baseline_mode = "aug" # "zeros", "aug"
result_columns = ['batch_index', 'explainer', 'metric', 'area', 'comp', 'suff']
output_file = open(os.path.join(result_folder, "batch_interpretation_results.csv"), 'w')
output_file.write(','.join(result_columns))

progress_bar = tqdm(
    enumerate(dataloader), total=len(dataloader), disable=False
)
for batch_index, (batch_x, batch_y, batch_x_mark, batch_y_mark) in progress_bar:
    batch_x = batch_x.float().to(exp.device)
    batch_y = batch_y.float().to(exp.device)

    batch_x_mark = batch_x_mark.float().to(exp.device)
    batch_y_mark = batch_y_mark.float().to(exp.device)
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float()
    # outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    
    inputs = batch_x
    # baseline must be a scaler or tuple of tensors with same dimension as input
    if baseline_mode=='zeros': baselines = torch.zeros_like(inputs)
    else: baselines = torch.mean(inputs, axis=0).repeat(inputs.shape[0], 1, 1).float()
    additional_forward_args = (batch_x_mark, dec_inp, batch_y_mark)

    # get attributions
    for name in explainers:
        explainer = explainers_map[name]
        attr = compute_attr(
            inputs, baselines, explainer, additional_forward_args, args
        )
    
        # get scores
        for area in areas:
            for metric_name, metric in expl_metric_map.items():
                error_comp = metric(
                    model, inputs=inputs, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=True
                )
                
                error_suff = metric(
                    model, inputs=inputs, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=False
                )
           
                result_row = [batch_index, name, metric_name, area, error_comp, error_suff]
                # print(result_row)
                output_file.write("\n" + ','.join([str(r) for r in result_row]))
                results.append(result_row)
        
        output_file.flush()
    # break
output_file.close()

## Output

In [ ]:
results_df = pd.DataFrame(results, columns=result_columns)
results_df = results_df.groupby(['explainer', 'metric', 'area'])[['comp', 'suff']].aggregate('mean').reset_index()
results_df.round(6).to_csv(os.path.join(result_folder, 'interpretation_results.csv'), index=False)
print(results_df)

# TSR

In [ ]:
results = []
baseline_mode = "aug" # "zeros", "aug"
result_columns = ['batch_index', 'explainer', 'metric', 'area', 'comp', 'suff']
output_file = open(os.path.join(result_folder, "batch_interpretation_results_tsr.csv"), 'w')
output_file.write(','.join(result_columns))

progress_bar = tqdm(
    enumerate(dataloader), total=len(dataloader), disable=False
)
for batch_index, (batch_x, batch_y, batch_x_mark, batch_y_mark) in progress_bar:
    batch_x = batch_x.float().to(exp.device)
    batch_y = batch_y.float().to(exp.device)

    batch_x_mark = batch_x_mark.float().to(exp.device)
    batch_y_mark = batch_y_mark.float().to(exp.device)
    # decoder input
    dec_inp = torch.zeros_like(batch_y[:, -exp.args.pred_len:, :]).float()
    dec_inp = torch.cat([batch_y[:, :exp.args.label_len, :], dec_inp], dim=1).float()
    # outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    
    inputs = batch_x
    # baseline must be a scaler or tuple of tensors with same dimension as input
    if baseline_mode=='zeros': baselines = torch.zeros_like(inputs)
    else: baselines = torch.mean(inputs, axis=0).repeat(inputs.shape[0], 1, 1).float()
    additional_forward_args = (batch_x_mark, dec_inp, batch_y_mark)

    # get attributions
    for name in explainers:
        explainer = explainers_map[name]
        attr = compute_tsr_attr(
            inputs, baselines, explainer, additional_forward_args, args
        )
    
        # get scores
        for area in areas:
            for metric_name, metric in expl_metric_map.items():
                error_comp = metric(
                    model, inputs=inputs, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=True
                )
                
                error_suff = metric(
                    model, inputs=inputs, 
                    attributions=attr, baselines=baselines, 
                    additional_forward_args=additional_forward_args,
                    topk=area, mask_largest=False
                )
           
                result_row = [batch_index, name, metric_name, area, error_comp, error_suff]
                # print(result_row)
                output_file.write("\n" + ','.join([str(r) for r in result_row]))
                results.append(result_row)
        
        output_file.flush()
    # break
    gc.collect()
output_file.close()

In [ ]:
tsr_results_df = pd.DataFrame(results, columns=result_columns)
tsr_results_df = tsr_results_df.groupby(['explainer', 'metric', 'area'])[['comp', 'suff']].aggregate('mean').reset_index()
tsr_results_df.round(6).to_csv(os.path.join(result_folder, 'interpretation_results_tsr.csv'), index=False)
print(tsr_results_df)

In [ ]:
explainer = FeatureAblation(model)
actual_attr = compute_attr(inputs, baselines, explainer, additional_forward_args, args)

In [ ]:
# batch x seq_len
time_attr = torch.zeros((inputs.shape[0], args.seq_len), dtype=float)

for t in tqdm(range(args.seq_len)):
    new_inputs = inputs.clone() 
    # batch x seq_len x features
    new_inputs[:, t] = 0 # test with new_inputs[:, :t+1] and other mastking

    new_attr_per_time = compute_attr(
        new_inputs, baselines, explainer, 
        additional_forward_args, args
    )
    
    # sum the attr difference for each input in the batch
    # batch x seq_len x features -> batch
    time_attr[:, t] = (actual_attr - new_attr_per_time
        ).abs().sum(axis=(1, 2))

In [ ]:
# for each input in the batch, normalize along the time axis
time_attr = torch.nn.functional.normalize(time_attr, p=1, dim=1)
# find median along the time axis
mean_time_importance = np.quantile(time_attr, .55, axis=1)   

In [ ]:
n_features = inputs.shape[-1]
input_attr = torch.zeros((inputs.shape[0], n_features), dtype=float)
time_scaled_attr = torch.zeros_like(actual_attr)

for t in tqdm(range(args.seq_len)):
    # if time_attr[t] < mean_time_importance:
    #     featureContibution = torch.ones(input_attr, dtype=float)/n_features
    for f in range(n_features):
        new_inputs = inputs.clone() # batch x seq_len x features
        new_inputs[:, t, f] = 0
        
        attr = compute_attr(
            new_inputs, baselines, explainer, 
            additional_forward_args, args
        )
        input_attr[:, f] = (actual_attr - attr).abs().sum(axis=(1, 2))
    
    input_attr = torch.nn.functional.normalize(input_attr, p=1, dim=1)
    
    for f in range(n_features):
        time_scaled_attr[:, t, f] = time_attr[:, t] * input_attr[:, f]

In [ ]:
results = []
name = 'feature_ablation'
for area in areas:
    for metric_name in ['mae', 'mse']:
        metric = getattr(tint.metrics, metric_name)
        error_comp = metric(
            model, inputs=inputs, 
            attributions=time_scaled_attr, baselines=baselines, 
            additional_forward_args=additional_forward_args,
            topk=area, mask_largest=True
        )

        error_suff = metric(
            model, inputs=inputs, 
            attributions=time_scaled_attr, baselines=baselines, 
            additional_forward_args=additional_forward_args,
            topk=area, mask_largest=False
        )
        result_row = [0, name, metric_name, area, error_comp, error_suff]
        results.append(result_row)

In [ ]:
tsr_results_df = pd.DataFrame(results, columns=result_columns)
tsr_results_df = tsr_results_df.groupby(['explainer', 'metric', 'area'])[['comp', 'suff']].aggregate('mean').reset_index()
# results_df.round(6).to_csv(os.path.join(result_folder, 'interpretation_results.csv'), index=False)
print(tsr_results_df)

In [ ]:
results_df[results_df['explainer']=='feature_ablation']

# Others

In [ ]:
explainer.

In [ ]:
explainer.get_name()

In [ ]:
explainer = Occlusion(model)
scores = explainer.attribute(
    inputs=inputs, baselines=baselines, sliding_window_shapes = (1,1),
    additional_forward_args=additional_forward_args
)

In [ ]:
scores.shape

In [ ]:
mae_error = mae(
    model, inputs=inputs, 
    attributions=attr, baselines=baselines, 
    additional_forward_args=additional_forward_args,
    topk=0.2
)

In [ ]:
# temporal_mask = torch.zeros_like(batch_x, dtype=int)
# for t in range(batch_x.shape[1]):
#     temporal_mask[:, t] = t

# explainer = FeatureAblation(model)
# time_score = explainer.attribute(
#     inputs=(batch_x),
#     baselines=(batch_x*0),
#     additional_forward_args=(batch_x_mark, dec_inp, batch_y_mark),
#     target=0,
#     feature_mask=temporal_mask
# )
# print(score.shape)

In [ ]:
# from tint.attr import Occlusion

# temporal_mask = torch.zeros(size=(1, *batch_x.shape[1:]), dtype=int)
# for t in range(batch_x.shape[1]):
#     temporal_mask[:, t, :] = t

# explainer = Occlusion(model)
# time_score = explainer.attribute(
#     inputs=(batch_x),
#     baselines=(batch_x*0),
#     additional_forward_args=(batch_x_mark, dec_inp, batch_y_mark),
#     sliding_window_shapes=(1, 1)
#     # feature_mask=temporal_mask.to(exp.device)
# )
# print(time_score.shape)